In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
project_folder = r"C:\Users\harsh\OneDrive\Desktop\Job\Temp resume\Avelo\Data\Data"  # <--- change this
db_path = os.path.join(project_folder, "avelo_routes.db")

In [3]:
db1b_folder = os.path.join(project_folder, "DB1B")
t100_folder = os.path.join(project_folder, "T100")

In [4]:
conn = sqlite3.connect(db_path)

In [5]:
db1b_files = [f for f in os.listdir(db1b_folder) if f.startswith("T_DB1B_MARKET") and f.endswith(('.xlsx', '.xls', '.csv'))]
db1b_frames = []

for file in db1b_files:
    path = os.path.join(db1b_folder, file)
    print(f"[DB1B] Processing: {file}")
    
    if file.endswith(".csv"):
        df = pd.read_csv(path)
    elif file.endswith(".xlsx"):
        df = pd.read_excel(path, engine='openpyxl')
    elif file.endswith(".xls"):
        df = pd.read_excel(path, engine='xlrd')
    else:
        continue

    df.columns = df.columns.str.upper().str.strip()
    if "REPORTING_CARRIER" in df.columns:
        df = df[df["REPORTING_CARRIER"] == "XP"]
        keep_cols = [
            "YEAR", "QUARTER", "ORIGIN", "DEST", "REPORTING_CARRIER",
            "PASSENGERS", "MARKET_FARE", "MARKET_DISTANCE"
        ]
        df = df[[col for col in keep_cols if col in df.columns]]
        db1b_frames.append(df)

if db1b_frames:
    db1b_all = pd.concat(db1b_frames, ignore_index=True)
    db1b_all.to_sql("db1b_market", conn, if_exists="replace", index=False)
    print(f" DB1B: Loaded {len(db1b_all)} rows into 'db1b_market'")
else:
    print(" No Avelo data found in DB1B files.")

[DB1B] Processing: T_DB1B_MARKET 2023 Q1.csv
[DB1B] Processing: T_DB1B_MARKET 2023 Q2.csv
[DB1B] Processing: T_DB1B_MARKET 2023 Q3.csv
[DB1B] Processing: T_DB1B_MARKET 2023 Q4.csv
[DB1B] Processing: T_DB1B_MARKET 2024 Q1.csv
[DB1B] Processing: T_DB1B_MARKET 2024 Q2.csv
[DB1B] Processing: T_DB1B_MARKET 2024 Q3.csv
[DB1B] Processing: T_DB1B_MARKET 2024 Q4.csv
[DB1B] Processing: T_DB1B_MARKET 2025 Q1.csv
✅ DB1B: Loaded 162865 rows into 'db1b_market'


In [6]:
t100_files = [f for f in os.listdir(t100_folder) if f.startswith("T_T100D_SEGMENT") and f.endswith(('.xlsx', '.xls', '.csv'))]
t100_frames = []

for file in t100_files:
    path = os.path.join(t100_folder, file)
    print(f"[T100] Processing: {file}")
    
    if file.endswith(".csv"):
        df = pd.read_csv(path)
    elif file.endswith(".xlsx"):
        df = pd.read_excel(path, engine='openpyxl')
    elif file.endswith(".xls"):
        df = pd.read_excel(path, engine='xlrd')
    else:
        continue

    df.columns = df.columns.str.upper().str.strip()
    df = df[df["UNIQUE_CARRIER"] == "XP"]
    keep_cols = [
        "YEAR", "QUARTER", "MONTH", "UNIQUE_CARRIER", "ORIGIN", "DEST",
        "PASSENGERS", "SEATS", "DEPARTURES_PERFORMED", "DISTANCE"
    ]
    df = df[[col for col in keep_cols if col in df.columns]]
    t100_frames.append(df)

if t100_frames:
    t100_all = pd.concat(t100_frames, ignore_index=True)
    t100_all.to_sql("t100_segment", conn, if_exists="replace", index=False)
    print(f" T100: Loaded {len(t100_all)} rows into 't100_segment'")
else:
    print("No Avelo data found in T100 files.")

[T100] Processing: T_T100D_SEGMENT_US_CARRIER_ONLY 2023.csv


C:\Users\harsh\AppData\Local\Temp\ipykernel_23652\4063606125.py:9: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


[T100] Processing: T_T100D_SEGMENT_US_CARRIER_ONLY 2024.csv


C:\Users\harsh\AppData\Local\Temp\ipykernel_23652\4063606125.py:9: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


[T100] Processing: T_T100D_SEGMENT_US_CARRIER_ONLY 2025.csv
✅ T100: Loaded 6093 rows into 't100_segment'


In [7]:
conn.close()
print(f" ]All data written to: {db_path}")

✅ All data written to: C:\Users\harsh\OneDrive\Desktop\Job\Temp resume\Avelo\Data\Data\avelo_routes.db
